# നെയിംഡ് എന്റിറ്റി റികഗ്നിഷൻ (NER)

ഈ നോട്ട്‌ബുക്ക് [AI for Beginners Curriculum](http://aka.ms/ai-beginners) നിന്നാണ്.

ഈ ഉദാഹരണത്തിൽ, നാം [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) ഡാറ്റാസെറ്റിൽ നിന്നുള്ള NER മോഡൽ എങ്ങനെ ട്രെയിൻ ചെയ്യാമെന്ന് പഠിക്കും. തുടക്കമാക്കുന്നതിന് മുമ്പ്, ദയവായി [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) ഫയൽ നിലവിലെ ഡയറക്ടറിയിലേക്ക് ഡൗൺലോഡ് ചെയ്യുക.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## ഡാറ്റാസെറ്റ് തയ്യാറാക്കൽ

ഡാറ്റാസെറ്റ് ഒരു ഡാറ്റാഫ്രെയിമിലേക്ക് വായിക്കുന്നതിൽ നിന്ന് നാം ആരംഭിക്കും. Pandas ഉപയോഗിച്ച് കൂടുതൽ അറിയാൻ ആഗ്രഹിക്കുന്നുവെങ്കിൽ, ഞങ്ങളുടെ [Data Science for Beginners](http://aka.ms/datascience-beginners) ലെ [ഡാറ്റാ പ്രോസസ്സിംഗ് പാഠം](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) സന്ദർശിക്കുക.


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


നമുക്ക് വ്യത്യസ്ത ടാഗുകൾ കണ്ടെത്തി ടാഗുകൾ ക്ലാസ് നമ്പറുകളായി മാറ്റാൻ ഉപയോഗിക്കാവുന്ന ലുക്കപ്പ് ഡിക്ഷണറികൾ സൃഷ്ടിക്കാം:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

ഇപ്പോൾ നമുക്ക് വാക്കുകളുടെ സമാഹാരത്തോടും അതേപോലെ ചെയ്യേണ്ടതുണ്ട്. ലളിതത്വത്തിനായി, വാക്കുകളുടെ ആവൃത്തി പരിഗണിക്കാതെ വാക്കുകളുടെ സമാഹാരം സൃഷ്ടിക്കാം; യഥാർത്ഥ ജീവിതത്തിൽ നിങ്ങൾക്ക് Keras വെക്ടറൈസർ ഉപയോഗിച്ച് വാക്കുകളുടെ എണ്ണം പരിമിതപ്പെടുത്താൻ ആഗ്രഹിക്കാം.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

നമുക്ക് പരിശീലനത്തിനായി വാക്യങ്ങളുടെ ഒരു ഡാറ്റാസെറ്റ് സൃഷ്ടിക്കേണ്ടതാണ്. മൗലിക ഡാറ്റാസെറ്റിലൂടെ ലൂപ്പ് ചെയ്ത് എല്ലാ വ്യക്തിഗത വാക്യങ്ങളും `X` (വാക്കുകളുടെ പട്ടികകൾ) ആയും `Y` (ടോക്കണുകളുടെ പട്ടിക) ആയും വേർതിരിക്കാം:


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


ഇപ്പോൾ നാം എല്ലാ വാക്കുകളും ടോക്കണുകളും വെക്ടറൈസ് ചെയ്യും:


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

സൗകര്യത്തിനായി, എല്ലാ വാക്യങ്ങളും പരമാവധി നീളത്തിലേക്ക് 0 ടോക്കണുകൾ ചേർത്ത് പാഡ് ചെയ്യും. യഥാർത്ഥ ജീവിതത്തിൽ, നാം കൂടുതൽ ബുദ്ധിമുട്ടുള്ള തന്ത്രം ഉപയോഗിച്ച്, ഒരു മിനിബാച്ചിനുള്ളിൽ മാത്രമേ സീക്വൻസുകൾ പാഡ് ചെയ്യൂ.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## ടോക്കൺ ക്ലാസിഫിക്കേഷൻ നെറ്റ്‌വർക്ക് നിർവചിക്കൽ

ടോക്കൺ ക്ലാസിഫിക്കേഷനായി രണ്ട് പാളി ബൈഡയറക്ഷണൽ LSTM നെറ്റ്‌വർക്ക് ഉപയോഗിക്കും. അവസാന LSTM പാളിയുടെ ഓരോ ഔട്ട്പുട്ടിനും ഡെൻസ് ക്ലാസിഫയർ പ്രയോഗിക്കാൻ, ഓരോ ഘട്ടത്തിലും LSTM-യുടെ ഔട്ട്പുട്ടുകൾക്ക് സമാനമായ ഡെൻസ് ലെയർ പുനരാവർത്തിപ്പിക്കുന്ന `TimeDistributed` ഘടന ഉപയോഗിക്കും:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

ഇവിടെ ശ്രദ്ധിക്കുക, ഞങ്ങൾ നമ്മുടെ ഡാറ്റാസെറ്റിനായി `maxlen` വ്യക്തമാക്കുകയാണ് - നെറ്റ്‌വർക്ക് വ്യത്യസ്ത നീളമുള്ള സീക്വൻസുകൾ കൈകാര്യം ചെയ്യാൻ കഴിയണമെങ്കിൽ, നെറ്റ്‌വർക്ക് നിർവചിക്കുമ്പോൾ കുറച്ച് കൂടുതൽ ബുദ്ധിമുട്ടുള്ള രീതിയിലാണ് വേണ്ടത്.

ഇപ്പോൾ മോഡൽ പരിശീലിപ്പിക്കാം. വേഗതക്കായി, നാം ഒരു epoch മാത്രമേ പരിശീലിപ്പിക്കൂ, പക്ഷേ നിങ്ങൾക്ക് കൂടുതൽ സമയം പരിശീലിപ്പിക്കാൻ ശ്രമിക്കാം. കൂടാതെ, നിങ്ങൾക്ക് ഡാറ്റാസെറ്റിന്റെ ചില ഭാഗം പരിശീലന ഡാറ്റാസെറ്റായി വേർതിരിച്ച് വാലിഡേഷൻ കൃത്യത നിരീക്ഷിക്കാൻ ആഗ്രഹിക്കാം.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## ഫലം പരീക്ഷിക്കൽ

ഇപ്പോൾ നമുക്ക് ഒരു സാമ്പിൾ വാക്യത്തിൽ നമ്മുടെ എന്റിറ്റി തിരിച്ചറിയൽ മോഡൽ എങ്ങനെ പ്രവർത്തിക്കുന്നു എന്ന് നോക്കാം:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## പ്രധാനപ്പെട്ട കാര്യങ്ങൾ

സാധാരണ LSTM മോഡലും NER-ൽ യുക്തിസഹമായ ഫലങ്ങൾ കാണിക്കുന്നു. എന്നാൽ, കൂടുതൽ മികച്ച ഫലങ്ങൾ നേടാൻ, BERT പോലുള്ള വലിയ പ്രീ-ട്രെയിൻ ചെയ്ത ഭാഷാ മോഡലുകൾ ഉപയോഗിക്കാം. Huggingface Transformers ലൈബ്രറി ഉപയോഗിച്ച് BERT NER-ക്ക് ട്രെയിൻ ചെയ്യുന്നത് [ഇവിടെ](https://huggingface.co/course/chapter7/2?fw=pt) വിശദീകരിച്ചിരിക്കുന്നു.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖ അധികാരപരമായ ഉറവിടമായി കണക്കാക്കപ്പെടണം. നിർണായക വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
